# verilerin lineer regresyon ile tahmin edilmesi

In [2]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [3]:
df = pd.read_csv('data_cleaned.csv')

In [4]:
#değişken türlerini düzenli bir biçimde tanımladık
df['İlçe'] = df['İlçe'].str.strip()
df['Mahalle'] = df['Mahalle'].str.strip()
df['İlçe'] = df['İlçe'].astype('category')
df['Mahalle'] = df['Mahalle'].astype('category')
df['Oda Sayısı'] = df['Oda Sayısı'].astype('int')
df['m^2'] = df['m^2'].astype('int')
df['Bina Yaşı'] = df['Bina Yaşı'].astype('int')
df['Kat'] = df['Kat'].astype('int')
df['Fiyat'] = df['Fiyat'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8512 entries, 0 to 8511
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   İlçe        8512 non-null   category
 1   Mahalle     8512 non-null   category
 2   Oda Sayısı  8512 non-null   int64   
 3   m^2         8512 non-null   int64   
 4   Bina Yaşı   8512 non-null   int64   
 5   Kat         8512 non-null   int64   
 6   Fiyat       8512 non-null   int64   
dtypes: category(2), int64(5)
memory usage: 369.3 KB


In [5]:
categorical_features = ['İlçe', 'Mahalle']
numerical_features = ['Oda Sayısı', 'm^2','Bina Yaşı', 'Kat']

In [6]:
#veri ön işleme pipeline'ı oluşuturuluyor. amaç hem sayısal hem de kategorik değişkenlere farklı dönüşün işlemleri uygulamak
#column transformer'ın amacı farklı sütun türlerine (nümerik ve kategorik), birden fazla dönüşüm uygulamak için
#sayısal değerleri 1 ve 0 arasında standardize ediyoruz. 
#kategorik sütunlara one hot encoding uygulanır, handle unknown ise eğitim sırasında karşılaşılmayan veri testte karşımıza çıkarsa hata almamak içind
full_pipeline = ColumnTransformer([
    ('num', StandardScaler(), numerical_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

In [7]:
#makine öğrenimi algoritması olarak kullanacağımız lineer regresyon'da bağımlı ve bağımsız değişkenleri atadık
X = df.drop('Fiyat', axis=1)
y = df['Fiyat']


In [8]:
#veriyi eğitim ve test setlerine böldük
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
#pipeline, veri hazırlığı ve modelleme işlemlerini bir araya getirir.
#amacı veri ön işleme ve modelleme adımlarını zincirlemek
#birinci adımda, oluşturmuş olduğumuz full_pipeline ile veri ön işleme yapılır
#sayısal sütunlar standardize edilir, kategorik sütunlar one hot encoding e dönüştürülür
#modeli lineer regresyon ile kuracağımızı parametre olarak girdik
#özetle pipeline, preparation ve model işlemlerinin özellik ve parametrelerini aldı, model için bir boru hattı oluşturdu,modelin değişkenleri ve değerleri atanıp model adlı bir değişkene atandı
model = Pipeline([
    ('preparation', full_pipeline),
    ('model', LinearRegression())
])

In [10]:
model.fit(X_train, y_train)

Pipeline(steps=[('preparation',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Oda Sayısı', 'm^2',
                                                   'Bina Yaşı', 'Kat']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['İlçe', 'Mahalle'])])),
                ('model', LinearRegression())])

In [11]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

In [12]:
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R^2: {r2}")

MSE: 18538607.130909253
RMSE: 4305.648282304217
R^2: 0.6555908810665734


In [13]:
feature_importances = model.named_steps['model'].coef_
print(len(feature_importances))
print(feature_importances)

397
[-1.51540741e+02  2.77248274e+03 -2.57704521e+03  8.47659547e+02
 -2.53294460e+03  4.24996577e+02  6.62170483e+02 -3.43944493e+02
 -4.91620479e+03  8.45952540e+02  3.96725436e+03 -2.00370923e+03
 -4.94505643e+03 -3.05864964e+03  4.94383317e+03  3.33673156e+03
  8.91399962e+02  1.71569535e+02 -6.81546802e+03 -8.81264658e+01
  8.13220901e+02  3.14359301e+03  1.20677779e+04 -6.56439631e+03
  1.03580248e+03 -2.45301324e+03 -3.56802999e+03  2.83681537e+03
 -1.54526606e+03  4.71794179e+02  3.81286626e+03 -2.14310559e+03
 -6.36144547e+03  1.48490305e+03  6.80299124e+03 -1.52829147e+03
  4.36471076e+01 -5.31599674e+03 -4.20479541e+03 -5.21587397e+02
  6.09697461e+02 -1.19052319e+04  1.71569535e+02  3.26164181e+01
  1.06178660e+04 -2.61633405e+03 -8.80305650e+02 -2.58240297e+03
 -4.02389521e+03 -5.21818693e+02  3.32874054e+03 -1.03757444e+04
 -6.63506550e+03 -1.17996206e+03  1.46918144e+03 -5.37351062e+03
 -5.07325404e+02 -4.26197078e+03  2.18637367e+03 -2.32298705e+03
 -4.66793032e+03  4.4

In [14]:
print("Numerical Features")
for i in range(len(numerical_features)):
    print(numerical_features[i], feature_importances[i])

Numerical Features
Oda Sayısı -151.5407409327541
m^2 2772.4827371090228
Bina Yaşı -2577.0452055418364
Kat 847.6595470164476


In [15]:
print("Categorical Features")
for i in range(len(categorical_features)):
    for j in range(len(model.named_steps['preparation'].transformers_[1][1].categories_[i])):
        print(model.named_steps['preparation'].transformers_[1][1].categories_[i][j], feature_importances[len(numerical_features) + j])

Categorical Features
Akyurt -2532.944604417418
Altındağ 424.99657732017016
Ayaş 662.1704830881702
Beypazarı -343.94449302133614
Elmadağ -4916.204792719714
Etimesgut 845.9525403136007
Gölbaşı 3967.2543560701706
Güdül -2003.7092308055949
Haymana -4945.056428952708
Kahramankazan -3058.64963747428
Keçiören 4943.833169309296
Kızılcahamam 3336.731564295862
Mamak 891.399961595757
Nallıhan 171.569534829554
Polatlı -6815.468021771311
Pursaklar -88.12646580528225
Sincan 813.2209009517697
Yenimahalle 3143.5930081482434
Çankaya 12067.777890133264
Çubuk -6564.396310859504
100. Yıl -2532.944604417418
19 Mayıs 424.99657732017016
23 Nisan 662.1704830881702
25 Mart -343.94449302133614
29 Ekim -4916.204792719714
30 Ağustos 845.9525403136007
Abidinpaşa 3967.2543560701706
Adnan Menderes -2003.7092308055949
Ahi Evran -4945.056428952708
Ahi Evran OSB -3058.64963747428
Ahi Mesut 4943.833169309296
Ahlatlıbel 3336.731564295862
Akdere 891.399961595757
Akpınar 171.569534829554
Aktepe -6815.468021771311
Akşemsett

In [16]:
new_data = pd.DataFrame({
    'İlçe': ['Gölbaşı'],
    'Mahalle': ['Bahçelievler'],
    'Oda Sayısı' : [3],
    'm^2': [90],
    'Bina Yaşı': [10],
    'Kat': [-1]
})

print(model.predict(new_data))

[21760.24500768]


In [17]:
print(df[(df['İlçe'] == 'Gölbaşı') & (df['Mahalle'] == 'Bahçelievler') ])


         İlçe       Mahalle  Oda Sayısı  m^2  Bina Yaşı  Kat  Fiyat
546   Gölbaşı  Bahçelievler           4  160          5    2  25000
547   Gölbaşı  Bahçelievler           4  160          5    2  25000
756   Gölbaşı  Bahçelievler           4  165         11    3  29500
1002  Gölbaşı  Bahçelievler           2   70         10    3  17000
1247  Gölbaşı  Bahçelievler           4  142         20    3  16000
1741  Gölbaşı  Bahçelievler           2   55         20    2  14500
2047  Gölbaşı  Bahçelievler           2   60          0    3  27000
2202  Gölbaşı  Bahçelievler           4  130         16    3  19000
2307  Gölbaşı  Bahçelievler           4  110         11    1  19500
2563  Gölbaşı  Bahçelievler           4  115         10    1  22500
2728  Gölbaşı  Bahçelievler           2   65         15    4  20000
2853  Gölbaşı  Bahçelievler           4  130         16    4  21000
3116  Gölbaşı  Bahçelievler           4  120         11    1  21000
3844  Gölbaşı  Bahçelievler           2   55    

In [19]:
def tolerance_r2(y_true, y_pred, tolerance):
    residuals = y_pred - y_true
    residuals[np.abs(residuals) <= tolerance] = 0
    ssr = np.sum(residuals**2)
    sst = np.sum((y_true - np.mean(y_true))**2)
    return 1 - (ssr / sst)

def tolerance_percentage_r2(y_true, y_pred, tolerance):
    residuals = y_pred - y_true
    residuals[(np.abs(residuals) / y_true) <= tolerance] = 0
    ssr = np.sum(residuals**2)
    sst = np.sum((y_true - np.mean(y_true))**2)
    return 1 - (ssr / sst)

In [28]:
print(r2_score(y_test, y_pred))
print(tolerance_r2(y_test, y_pred, 10000))
print(tolerance_percentage_r2(y_test, y_pred, 0.5))

0.6555908810665734
0.9012574051551538
0.9637375652708551
